### Chapter IV - WWW, Wiki and Online social networks.

#### This __exercise__ notebook is taken from the Python 2 notebook for Ch. 4 of Caldarelli-Cheesa's textbook (CC).

The challege questions at the bottom are solved in the `solution` version of this notebook.

In [ ]:
import numpy as np

import matplotlib.pyplot as plt

import networkx as nx


#### Get data from The Laboratory for Web Algorithmics

#### This is the page with the datasets: http://law.di.unimi.it/datasets.php

It is possible to download a network in a WebGraph format that is a compressed binary format.

The project provides various clients to extract the network strcture, in Java, C++ and in Python, py-web-graph: http://webgraph.di.unimi.it/.

In particular we got the graph and the related urls associated to each node of the .eu domain in 2005: http://law.di.unimi.it/webdata/eu-2005/.

 We exctracted the graph in a form of an edge list and we also got the file with the list of urls in the same order of the node_id

##### For Colab execution:

Please upload the two data files shown below to your `sample_data` folder in Colab.

In [ ]:
ARCSFILE = './sample_data/eu-2005_1M.arcs'

URLSFILE = './sample_data/eu-2005.urls'

In [ ]:
#retrieve just the portion of the first 1M edges of the .eu domain
#crawled in 2005
eu_DG = nx.read_edgelist(ARCSFILE, create_using = nx.DiGraph())

#generate the dictionary of node_id -> urls
file_urls = open(URLSFILE)

count = 0

dic_nodid_urls = {}

while True:
    next_line = file_urls.readline()

    if not next_line:
        break

    dic_nodid_urls[str(count)] = next_line[ :-1]
    count = count+1

file_urls.close()

#generate the strongly connected component
scc = [(len(c),c) for c in sorted( nx.strongly_connected_components \
                               (eu_DG), key=len, reverse=True)][0][1]

eu_DG_SCC = eu_DG.subgraph(scc)


In [ ]:
l = [e for e in eu_DG_SCC.edges]

What's in the data?

In [ ]:
l[ :5]

#### Retrieving data through the  [Twitter API](https://dev.twitter.com/docs) usign the [Twython](http://twython.readthedocs.org/en/latest/) module

This part is not in use anymore as the TwitterAPI does not generally serve data anymore: we get a `403` error.

Please proceed to the 'HITS algorithm' section below.

## Hits algorithm

##### Create a simple labeled network: the 'four triangles' network

In [ ]:
DG = nx.DiGraph()

DG.add_edges_from([('A','B'),('B','C'),('A','D'),
                   ('D','B'),('C','D'),('C','A')])

#plot the graph
nx.draw(DG, with_labels = True)

The network has a certain symmetry: each node has in-degree of 2 and out-degree of 1 or vice versa.


#### Direct implementation of the [HITS algorithm](https://en.wikipedia.org/wiki/HITS_algorithm) by [Kleinberg](https://en.wikipedia.org/wiki/Jon_Kleinberg).

In [ ]:
def HITS_algorithm(digraph: nx.DiGraph, K: int = 1000) -> tuple[dict, dict]:
    """
    :param digraph: A networkx DiGraph
    :param K: The K maximum number of iterations

    :return: Two dictionaries containing the hub and authority scores, resp.
    """
    auth_scores = {}
    hub_scores = {}

    for n in digraph.nodes():
        auth_scores[n] = 1.0
        hub_scores[n] = 1.0

    for k in range(K):
        norm = 0.0

        for n in digraph.nodes():
            auth_scores[n] = 0.0

            # REMINDER: a predecessor of a node n is a node m
            # such that there is a direct edge from m to n
            for p in digraph.predecessors(n):
                auth_scores[n] += hub_scores[p]

            norm += auth_scores[n]**2.0

        norm = norm**0.5

        for n in digraph.nodes():
            auth_scores[n] = auth_scores[n] / norm

        norm = 0.0

        for n in digraph.nodes():
            hub_scores[n] = 0.0

            for s in digraph.successors(n):
                hub_scores[n] += auth_scores[s]

            norm += hub_scores[n]**2.0

        norm = norm**0.5

        for n in digraph.nodes():
            hub_scores[n] = hub_scores[n] / norm

        return auth_scores, hub_scores

#### Let's put HITS to test.

In [ ]:
(auth, hub) = HITS_algorithm(DG, K=100)

print (auth)
print (hub)

### Q1.  Use built in hits function to find hub and authority scores.

Can you spot the differences in result?

In [ ]:
nx.draw_networkx(DG, with_labels = True)

# your solution here.

#### Adjacency matrix representation with basic operations

We refrain from using the standard `Numpy` methods for transposing and multiplying matrices.

In [ ]:
def matrix_transpose(M: list) -> list:

    M_out=[]

    for c in range(len(M[0])):

        M_out.append([])

        for r in range(len(M)):
            M_out[c].append(M[r][c])

    return M_out


def matrix_multiplication(M1: list, M2: list) -> list:

    M_out=[]

    for r in range(len(M1)):

        M_out.append([])

        for j in range(len(M2[0])):
            e=0.0

            for i in range(len(M1[r])):
                e+=M1[r][i]*M2[i][j]

            M_out[r].append(e)

    return M_out

Now, let's test the home-brew functions.

In [ ]:

adjacency_matrix1=[
                  [0,1,0,1],
                  [1,0,1,1],
                  [0,1,0,0]
                  ]

adjacency_matrix2 = matrix_transpose(adjacency_matrix1)

print ("Transpose adjacency matrix:", adjacency_matrix2)

res_mul = matrix_multiplication(adjacency_matrix1, adjacency_matrix2)

print ("Matrix multiplication:", res_mul)

Differently from the `Numpy` methods, our functions work with pure lists.

In [ ]:
type(res_mul)

### The Power-iterations algorithm: a direct implementation

In [ ]:
adjacency_matrix=[
                  [0,1,0,1],
                  [1,0,1,1],
                  [0,1,0,0],
                  [1,1,0,0]
                  ]
vector=[
        [0.21],
        [0.34],
        [0.52],
        [0.49]
        ]

# For small examples, few iterations will be needed.
C = 100

In [ ]:
for i in range(C):
    res = matrix_multiplication(adjacency_matrix, vector)

    norm_sq = 0.0

    for r in res:
        norm_sq = norm_sq+r[0]*r[0]

    vector = []

    for r in res:
         vector.append([r[0]/(norm_sq**0.5)])

print ("Maximum eigenvalue (in absolute value):", norm_sq**0.5)
print ("Eigenvector for the maximum eigenvalue:", vector)


#### Putting it all together: computing HITS for the WWW strongly-connected component of the `.eu` domain

In [ ]:
# Use operator.itemgetter(1) to sort the dictionary by value
import operator

In [ ]:
# Your solution here

#Please assign your results to lists sorted_auth and sorted_hub, respectively.




Below, uncomment as needed.

In [ ]:
#top ranking auth
print ("Top 5 by auth")

#for p in sorted_auth[:5]:
#    print (dic_nodid_urls[p[0]], p[1])

#top ranking hub
print ("Top 5 by hub")

#for p in sorted_hub[:5]:
#   print (dic_nodid_urls[p[0]], p[1])

### Q2. Run the built-in `nx.hits` function; can you spot the differences in result?

In [ ]:
# Your solution here

#Please assign your results to lists sorted_auth and sorted_hub, respectively.


Uncomment as needed.

In [ ]:
# print ("Top-5 auth nodes:")

# for p in sorted_auth[:5]:
#     print (dic_nodid_urls[p[0]], p[1])
#     print ("Top-5 hub nodes:")

# for p in sorted_hub[:5]:
#     print (dic_nodid_urls[p[0]], p[1])

#### Compute the PageRank

In [ ]:
def pagerank(graph, damping_factor = 0.85, max_iterations = 100, min_delta = 0.00000001):

    nodes = graph.nodes()

    graph_size = len(nodes)

    if graph_size == 0:
        return {}

    # itialize the page rank dict with 1/N for all nodes
    page_rank = dict.fromkeys(nodes, (1.0-damping_factor)*1.0/ graph_size)

    min_value = (1.0-damping_factor)/len(nodes)

    for _ in range(max_iterations):
        #total difference compared to last iteraction
        diff = 0

        # computes each node PageRank based on inbound links
        for node in nodes:
            rank = min_value

            for referring_page in graph.predecessors(node):
                rank += damping_factor * page_rank[referring_page]/ \
                len(list(graph.neighbors(referring_page)))

            diff += abs(page_rank[node] - rank)

            page_rank[node] = rank

        #stop if PageRank has converged
        if diff < min_delta:
            break

    return page_rank

#### The Networkx version of [PageRank](http://networkx.github.io/documentation/latest/reference/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html)

In [ ]:
G = nx.DiGraph()

G.add_edges_from([(1, 2), (2, 3), (3, 4), (3, 1), (4, 2)])
#plot the network

nx.draw(G, with_labels = True)

#our Page Rank algorithm
res_pr=pagerank(G, max_iterations = 10000, min_delta = 0.00000001, damping_factor = 0.85)
print (res_pr)

#Networkx Pagerank function
print (nx.pagerank(G,max_iter = 10000))

#### The Twitter Mention Network

Please skip this section as we don't access Twitter/X data anymore; proceed to he `Scwiki` section below.

#### Community detection I: the Karate Club

Let visualise a prediction of the community structure of the Karate Club network *before the split*.

ideally, we expect two communities (or *factions*), one around node 1 (the president of the club) and one around node 34 (the karate master).

We will operate the `Louvain` community detection algorithm<!-- .slide: data-fullscreen -->


In [ ]:
!pip install python-louvain

In [ ]:
import community

In [ ]:
DATAFILE = './sample_data/karate.dat'

In [ ]:
G = nx.read_edgelist(DATAFILE)

Run community detection. 

The Leuven algorithm, which we have not yet seen in class, has a random component, so it makes sense to run it a few times to seek a *clean* visualisation. 

Notice how, since we know what to expect from the Karate Club, we are defaulting on a sort of supervised version of the community detection problem.

In [ ]:
# try running it mopre than once to see how the results change
partition = community.best_partition(G)

Plot the result to see how close to the expected result we are.

In [ ]:


#plot the network
size = float(len(set(partition.values())))

pos = nx.spring_layout(G)

count = 0.

plt.axis('off')

for com in set(partition.values()) :
    
    count += 1
    
    list_nodes = [nodes for nodes in partition.keys() \
                  if partition[nodes] == com]
    
    nx.draw_networkx_nodes(G, pos, list_nodes, node_size = 300, \
                           node_color = str(count / size))
    
    nx.draw_networkx_labels(G,pos)

nx.draw_networkx_edges(G, pos, alpha=0.5, width=1)

plt.show()

In [ ]:
# save the result visualization

plt.savefig('./sample_data/karate_community.png', dpi=600)

#### Community Detection for the `Scwiki` network

Wikipedia is public and very connected, both internally (Wikipedia links) and externally (links to other sites).

It is interesting to see whether the links connecting two pages (lemmata of the encyclopedia) determine some community of concepts and, ultimately, a bottom-up, data/user-driven taxonomy of concepts, as in *curated* scientific taxonomies etc.

The present shape of the Wikipedia is available from [dumps.wikimedia.org](https://dumps.wikimedia.org/).

The data presented here is a tiny fragment of the *Sardininan* Wikipedia, which is developed for the spoken language of [Sardinia](https://en.wikipedia.org/wiki/Sardinia): 

[sc.wikipedia.org/](https://sc.wikipedia.org/)


#### References:

- for the structure of the *PageLinks* table files: [www.mediawiki.org/wiki?Manual:Page_links_table](https://www.mediawiki.org/wiki?Manual:Page_links_table)

- for the structure of the Page table files: [www.mediawiki.org/wiki?Manual:Page_table](https://www.mediawiki.org/wiki?Manual:Page_table)

In [ ]:
SCWIKI = './sample_data/scwiki_edgelist.dat'

TITLES = './sample_data/scwiki_page_titles.dat'

Warning: in `.eu` there are pages in the Sardinian language (and perhaps others) which require a specific coding on your own platform.

In [ ]:
#load the directed and undirected version og the scwiki graph
scwiki_pagelinks_net_dir = nx.read_edgelist(SCWIKI, create_using = nx.DiGraph())

scwiki_pagelinks_net = nx.read_edgelist(SCWIKI)

#load the page titles
dict_titles = {}

file_titles = open(TITLES, 'r', encoding='utf-8')

while True:
    next_line = file_titles.readline()

    if not next_line:
        break

    print (next_line.split()[0], next_line.split()[1])

    dict_titles[next_line.split()[0]] = next_line.split()[1]

file_titles.close()

Let's try to partition the Sardinian Wikipedia network into communities.

In [ ]:
partition = community.best_partition(scwiki_pagelinks_net)

Now a new network is created, which represents communities of concepts and their connections.

In [ ]:
#Generate representative nodes of the community structure
community_structure = nx.Graph()

diz_communities={}
diz_node_labels={}
diz_node_sizes={}
max_node_size=0

In [ ]:

for com in set(partition.values()) :
    diz_communities[com] = [nodes for nodes in partition.keys() \
                            if partition[nodes] == com]
    
    if len(diz_communities[com])>=200:
        if max_node_size<len(diz_communities[com]):
            max_node_size=len(diz_communities[com])
        print("community",com,len(diz_communities[com]), end=' ')
    
        sub_scwiki_dir = scwiki_pagelinks_net_dir.subgraph \
        (diz_communities[com])    
    
        res_pr=nx.pagerank(sub_scwiki_dir,max_iter=10000)
    
        sorted_pr=sorted(res_pr.items(), key=operator.itemgetter \
                         (1),reverse=True)

        print(dict_titles[sorted_pr[0][0]],sorted_pr[0][1])
    
        community_structure.add_node(com)

        diz_node_labels[com] = dict_titles[sorted_pr[0][0]]
    
        diz_node_sizes[com] = len(diz_communities[com])
       
#Generate edge weights according to the number of links
#among communities
max_edge_weight=0.0

for i1 in range(community_structure.number_of_nodes()-1):
    
    for i2 in range(i1+1,community_structure.number_of_nodes()):
        wweight=0.0
        
        for n1 in diz_communities[list(community_structure.nodes())[i1]]:
            for n2 in diz_communities[list(community_structure.nodes()) \
                                      [i2]]:
                if scwiki_pagelinks_net.has_edge(n1,n2):
                    wweight += 1.0
                    
        if wweight>100.0:
            if max_edge_weight<wweight:
                max_edge_weight=wweight
                
            community_structure.add_edge(list(community_structure. \
            nodes())[i1],list(community_structure.nodes())[i2], \
                                         weight=wweight)

#### Visualise the community of concepts

The Sardinian Wikipedia, seen as a network of pages, reveals a dual nature: one is the communities of *abstract concepts* such as history, language, adminstrative organisation etc. The other is the mapping of the island and its villages.

#### Installation section

Graphviz and pydot are known to have recurrent issues with installation, especially on Windows.

Test them out on your own platform before proceeding.

Reference page: [networkx](https://networkx.org/documentation/stable/reference/generated/networkx.drawing.nx_pydot.graphviz_layout.html)

In [ ]:
!pip install graphviz

In [ ]:
!pip install pydot

Test the simplest possible visualisation.

In [ ]:
LAYOUT = 'circo'

In [ ]:
G = nx.complete_graph(4)

pos_default = nx.nx_pydot.graphviz_layout(G)

pos_circo = nx.nx_pydot.graphviz_layout(G, prog=LAYOUT)

Select a visualisation layout, try them out; default is `neato`.<!-- .slide: data-fullscreen -->

- `circo`: arranges nodes in circles

- `dot`: hierarchical layout for DAGs

- `neato`: the spring-model layout, it uses force-directed positioning

- `fdp`: the same spring model but with *repulsive* force
  
- `sfdp`: multiscale version of the above, useful for large graphs
  
- `twopi`: the radial layout, it uses concentric circles

In [ ]:
LAYOUT = 'circo'

In [ ]:
pos = nx.nx_pydot.graphviz_layout(community_structure)

In [ ]:
# set up your favorite visualisation
node_size_factor = 2000.0

edge_weight_factor = 10.0

# axes not needed here
plt.axis('off')

In [ ]:


for n in community_structure.nodes():
    nx.draw_networkx_nodes(community_structure, pos, [n], node_size\
                           = node_size_factor*diz_node_sizes[n]/ \
                           max_node_size, node_color='Black')
    nx.draw_networkx_labels(community_structure,pos, font_color= \
                            'White',axis='off')

for e in community_structure.edges():
    nx.draw_networkx_edges(community_structure,pos,[e],alpha=0.5, \
                           width=edge_weight_factor* \
                           community_structure[e[0]][e[1]]['weight']\
                           /max_edge_weight)

plt.show()